### Generate Option Price Data Set

In [48]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
## Black Scholes Option Pricing Formula

def BS_Call(S,K,r,T,v,d=0):
    d1 = (np.log(float(S)/K)+((r-d)+v*v/2.)*T)/(v*np.sqrt(T))
    d2 = d1-v*np.sqrt(T)
    return S*np.exp(-d*T)*ss.norm.cdf(d1)-K*np.exp(-r*T)*ss.norm.cdf(d2)



In [50]:
## Build pandas dataframe of prices
S = np.arange(1, 100, 1)
K = np.arange(1, 100, 1)
r = 0.05
T = np.arange(0.05, 1, 0.1)
V = np.arange(0.05, 0.5, 0.04)

# Number of option prices = life begins at a million examples
no_of_options = len(S)*len(K)*len(T)*len(V)
no_of_options

1176120

In [51]:
prices = np.empty([no_of_options,6], dtype=float)

In [52]:
%%time
x = 0
for s in S:
    for k in K:
        for t in T:
            for v in V:
                prices[x,:] = [s,k,t,0.05,v,BS_Call(s,k,r,t,v)]
                x+=1

Wall time: 2min 54s


In [53]:
option_df = pd.DataFrame(index = range(no_of_options), columns = 
                         ['Stock', 
                          'Strike',
                          'Time',
                          'Risk_free',
                          'Volatility',
                          'Call Price'], data = prices )

In [57]:
option_df.tail(10)

,Stock,Strike,Time,Risk_free,Volatility,Call Price
1176110,99.0,99.0,0.95,0.05,0.13,7.520076
1176111,99.0,99.0,0.95,0.05,0.17,8.941301
1176112,99.0,99.0,0.95,0.05,0.21,10.389541
1176113,99.0,99.0,0.95,0.05,0.25,11.850355
1176114,99.0,99.0,0.95,0.05,0.29,13.316823
1176115,99.0,99.0,0.95,0.05,0.33,14.785075
1176116,99.0,99.0,0.95,0.05,0.37,16.252664
1176117,99.0,99.0,0.95,0.05,0.41,17.717881
1176118,99.0,99.0,0.95,0.05,0.45,19.179435
1176119,99.0,99.0,0.95,0.05,0.49,20.636284


### Build Keras Model

In [56]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [54]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [55]:
# Decide split for train and test set

m = 0.5
end = len(prices)
split = int(m*end)
split

588060

In [58]:
## Define X & Y
X_train = prices[:split,:5]
y_train = prices[:split,5:6]
X_test = prices[split:end, :5]
y_test = prices[split:end,5:6]


In [68]:
model = Sequential()
model.add(Dense(10, input_shape=(5,), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

In [69]:
model.compile(loss='mean_squared_error',
        optimizer='adam')

In [70]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
dropout_3 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_test, y_test, epochs=10, batch_size=10)

Epoch 1/10
588060/588060 [==============================] - 46s 78us/step - loss: 17.3742
Epoch 2/10
588060/588060 [==============================] - 45s 77us/step - loss: 6.5878
Epoch 3/10
588060/588060 [==============================] - 46s 78us/step - loss: 5.5490
Epoch 4/10
588060/588060 [==============================] - 45s 77us/step - loss: 5.4500
Epoch 5/10
588060/588060 [==============================] - 46s 78us/step - loss: 4.9470
Epoch 6/10
588060/588060 [==============================] - 46s 77us/step - loss: 4.6254
Epoch 7/10
588060/588060 [==============================] - 45s 77us/step - loss: 4.8065
Epoch 8/10
588060/588060 [==============================] - 45s 77us/step - loss: 4.9450
Epoch 9/10
555960/588060 [===========================>..] - ETA: 2s - loss: 4.7647

In [63]:
predictions = model.predict(X_train)

In [62]:
for x in range(50):
    print(predictions[x], y_train[x])

[3.296135] [0.00581406]
[3.316082] [0.00932863]
[3.336029] [0.01287316]
[3.355978] [0.01642639]
[3.375924] [0.01998314]
[3.395872] [0.02354151]
[3.4158182] [0.0271006]
[3.4357672] [0.03065993]
[3.4557142] [0.03421916]
[3.4756622] [0.0377781]
[3.4956093] [0.04133657]
[3.5155563] [0.04489443]
[3.3335295] [0.01200258]
[3.3534775] [0.0179086]
[3.3734236] [0.02396661]
[3.3933716] [0.03006926]
[3.4133186] [0.0361901]
[3.4332666] [0.04231931]
[3.4532137] [0.04845226]
[3.4731627] [0.05458638]
[3.4931107] [0.06072007]
[3.5130558] [0.06685219]
[3.5330038] [0.07298188]
[3.5529518] [0.07910846]
[3.370925] [0.01733611]
[3.390873] [0.02473422]
[3.410819] [0.03245155]
[3.430766] [0.04026384]
[3.450714] [0.04811498]
[3.470662] [0.055984]
[3.4906092] [0.06386103]
[3.5105562] [0.07174059]
[3.5305042] [0.0796192]
[3.5504513] [0.08749446]
[3.5703993] [0.09536454]
[3.5903473] [0.10322794]
[3.4083185] [0.02236124]
[3.4282665] [0.03085551]
[3.4482136] [0.03986594]
[3.4681616] [0.0490319]
[3.4881096] [0.05826